# Import

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd
import polars as pl
import tsfel
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

https://tsfel.readthedocs.io/en/latest/descriptions/get_started.html

In [ ]:
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']
DATA_FOLD = params['data_folder']

In [ ]:
df = pl.read_parquet(f'{DATA_FOLD}/{VERSION}/3.analysis/imputation_48/{DATASET}/tables/first_48_with_static_imputed_saits.parquet')

In [ ]:
pmsi_data = pl.read_parquet((f'{DATA_FOLD}/{VERSION}/2.clean_data/chu/static/pmsi_48.parquet'))

# Split train/test

In [ ]:
df_filter_pmsi = df.drop(['gender', 'age', 'admission_type']).join(pmsi_data, on='encounterId', how='inner').to_pandas()

In [ ]:
encounters_list = df_filter_pmsi['encounterId'].unique()

In [ ]:
train_encounters, test_encounters = train_test_split(encounters_list, test_size=0.3, random_state=42)

In [ ]:
print(train_encounters.shape)
print(test_encounters.shape)

In [ ]:
train_df = df_filter_pmsi[df_filter_pmsi['encounterId'].isin(train_encounters)]
test_df = df_filter_pmsi[df_filter_pmsi['encounterId'].isin(test_encounters)]

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df_10 = train_df[train_df['intervalle'] < 10]
test_df_10 = test_df[test_df['intervalle'] < 10]

In [ ]:
train_df

In [ ]:
train_temporal = train_df_10.iloc[:,2:6]
test_temporal = test_df_10.iloc[:,2:6]

# TSFEL Feature selections

In [ ]:
cfg = tsfel.get_features_by_domain(['statistical', 'temporal'])  # Extrait toutes les caractéristiques par domaine (statistiques, temporelles, spectrales)

In [ ]:
train_features = tsfel.time_series_features_extractor(config=cfg, timeseries=train_temporal, window_size=48)


In [ ]:
train_features.shape

In [ ]:
test_features = tsfel.time_series_features_extractor(config=cfg, timeseries=test_temporal, window_size=48)
test_features.shape

In [ ]:
test_features

In [ ]:
corr_features, Dataset_trainv1 = tsfel.correlated_features(train_features, drop_correlated=True)

In [ ]:
cols = Dataset_trainv1.columns

In [ ]:
selector = VarianceThreshold()

In [ ]:
selector.fit(Dataset_trainv1)

In [ ]:
colsV2=selector.get_feature_names_out(cols)

In [ ]:
colsV2

In [ ]:
colsV3 = [col for col in colsV2 if not (col.endswith('_Skewness') or col.endswith('_Kurtosis'))]

In [ ]:
Dataset_trainv2 = Dataset_trainv1[colsV3]
Dataset_testv2 = test_features[colsV3]

In [ ]:
train_static

In [ ]:
Dataset_trainv2

In [ ]:
# Normalising Features
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(Dataset_trainv2)
X_test = scaler.transform(Dataset_testv2)

# Merge static data

In [ ]:
train_static = train_df.drop_duplicates('encounterId').reset_index().drop(columns=['intervalle','heart_rate', 'spo2', 'fr', 'pam', 'index', 'DP', 'Libellé DP', 'utcInTime', 'utcOutTime',  'DtSortie'])
test_static = test_df.drop_duplicates('encounterId').reset_index().drop(columns=['intervalle','heart_rate', 'spo2', 'fr', 'pam', 'index', 'DP', 'Libellé DP', 'utcInTime', 'utcOutTime',  'DtSortie'])

In [ ]:
train_join = train_static.join(Dataset_trainv2)
test_join = test_static.join(Dataset_testv2)

# Prepare Data

In [ ]:
train_join.columns

In [ ]:
X_cat = ['gender', 'admission_type', 'cat_dp']
y_class = ['deces_pmsi']
y_reg = ['icu_los', 'hosp_los']
Y_feat = y_class+y_reg

In [ ]:
Y_feat

In [ ]:
X_train = train_join.copy().drop(columns=Y_feat)
X_test = test_join.copy().drop(columns=Y_feat)

Y_train_reg = train_join[y_reg].copy()
Y_train_class = train_join[y_class].copy()

Y_test_reg = test_join[y_reg].copy()
Y_test_class = test_join[y_class].copy()

In [ ]:
df_filter_pmsi['gender'].value_counts()

In [ ]:
le = preprocessing.LabelEncoder()

for feat in categorial_feat :
    print(feat)
    le.fit(df_filter_pmsi[feat])

    X_train[feat] = le.transform(X_train[feat])
    X_test[feat] = le.transform(X_test[feat])

In [ ]:
# Normalising Features
scaler = preprocessing.StandardScaler()
X_train.loc[:, ~X_train.columns.isin(categorial_feat)] = scaler.fit_transform(X_train.loc[:, ~X_train.columns.isin(categorial_feat)])
X_test.loc[:, ~X_test.columns.isin(categorial_feat)] = scaler.transform(X_test.loc[:, ~X_test.columns.isin(categorial_feat)])

In [ ]:
X_cols

In [ ]:
X_cols = X_train.iloc[:,5:].columns
X_train = X_train.iloc[:,5:].to_numpy()
X_test = X_test.iloc[:,5:].to_numpy()

In [ ]:
Y_train = Y_train_class.to_numpy()
Y_test = Y_test_class.to_numpy()

# Classification

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, Y_train_class)

In [ ]:
labels = ['survie', 'deces']

In [ ]:
y_train_resampled.sum()/len(y_train_resampled)

In [ ]:
classif = RandomForestClassifier()
# Train the classifier
classif.fit(X_train_resampled, y_train_resampled)

# Predict test data
y_test_predict = classif.predict(X_test)

# Get the classification report
accuracy = accuracy_score(Y_test, y_test_predict) * 100
print(classification_report(Y_test, y_test_predict, target_names=labels))
print("Accuracy: " + str(accuracy) + "%")

In [ ]:
# @title Confusion Matrix
cm = confusion_matrix(Y_test, y_test_predict)
df_cm = pd.DataFrame(cm, index=[i for i in labels], columns=[i for i in labels])
plt.figure()
ax = sns.heatmap(df_cm, cbar=False, cmap="BuGn", annot=True, fmt="d")
plt.setp(ax.get_xticklabels(), rotation=45)

plt.ylabel("True label", fontweight="bold", fontsize=18)
plt.xlabel("Predicted label", fontweight="bold", fontsize=18)
bottom, top = ax.get_ylim()
plt.show()

In [ ]:
import shap

# Appliquer les valeurs de Shapley
explainer = shap.TreeExplainer(classif)
shap_values = explainer.shap_values(X_train)

In [ ]:
plt_shap = shap.summary_plot(shap_values, #Use Shap values array
                             features=X_train, # Use training set features
                             feature_names=X_cols, #Use column names
                             show=False, #Set to false to output to folder
                             plot_size=(300,150)) # Change plot size
# Save my figure to a directory

plt.savefig("global_shapHAR.png")

In [ ]:

# Visualisation des contributions des caractéristiques
# Résumé des valeurs de Shapley pour chaque classe
for i, shap_val in enumerate(shap_values):
    print(f"Résumé pour la classe {i}:")
    shap.summary_plot(shap_val, X_train)

# T-SNE

In [ ]:
from openTSNE import TSNE


In [ ]:
tsne = TSNE(
    perplexity=500,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
)

In [ ]:
%time embedding_train = tsne.fit(X_train)

In [ ]:
embedding_train.shape

In [ ]:
y_train.shape

In [ ]:
# Visualize the results
plt.figure(figsize=(8, 6))
sns.scatterplot(x=embedding_train[:, 0], y=embedding_train[:, 1], edgecolor='k', alpha=0.7, hue=y_train)
plt.title("t-SNE Visualization")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.show()


In [ ]:
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D

tsne = TSNE(n_components=3, random_state=42, perplexity=500, n_iter=1000)
X_tsne_3d = tsne.fit_transform(X_train)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    X_tsne_3d[:, 0], X_tsne_3d[:, 1], X_tsne_3d[:, 2], 
    c='blue', edgecolor='k', alpha=0.7
)

# Ajouter des titres et des étiquettes
ax.set_title("Visualisation t-SNE en 3D")
ax.set_xlabel("Composante t-SNE 1")
ax.set_ylabel("Composante t-SNE 2")
ax.set_zlabel("Composante t-SNE 3")

# Afficher le graphique
plt.show()
